# Crawling data code

In [ ]:
import json

with open('processed_table.json') as f:
    data = json.load(f)
    
import sys
import requests
import urllib.request, urllib.error, urllib.parse
import os

print('original data has {} entries'.format(len(data)))
new_data = []
for i, d in enumerate(data):
    sys.stdout.write("finished {}/{} \r".format(i, len(data)))
    title = d['title']
    title = '_'.join(title.split(' '))    
    page = 'https://en.wikipedia.org/wiki/{}'.format(title)
        
    if len(d['data']) > 5 and len(d['data']) < 40 and len(d['data'][0]) >= 4:
        headers = set(d['header'])
        if len(headers) == len(d['header']):
            cols = len(d['header'])
            count = 0
            for g in d['data'][0]:
                if g[1] is not None:
                    count += 1
            if count < 0.3 * cols:
                continue
            
            # process if there are enough hyperlinks
            title = d['title']
            title = '_'.join(title.split(' '))
                
            if not os.path.exists('htmls/{}.html'.format(title)):
                try:
                    response = urllib.request.urlopen(page)
                    webContent = response.read()
                    f = open('htmls/{}.html'.format(title), 'wb')
                    f.write(webContent)
                    f.close()
                except Exception:
                    continue

            d['page'] = '{}.html'.format(title)
            new_data.append(d)

print('filtered data has {} entries'.format(len(new_data)))
with open('processed_table_with_page.json', 'w') as f:
    json.dump(new_data, f, indent=2)

In [ ]:
import os
from bs4 import BeautifulSoup
import sys
import json
import re
from multiprocessing import Pool
import multiprocessing

def process_link(text):
    tmp = []
    hrefs = []
    for t in text.find_all('a'):
        if len(t.get_text().strip()) > 0:
            if 'href' in t.attrs and t['href'].startswith('/wiki/'):
                tmp.append(t.get_text().strip())
                hrefs.append(t['href'])
            else:
                tmp.append(t.get_text().strip())
                hrefs.append('#')
    if all([_ == '#' for _ in hrefs]):
        return ','.join(tmp).strip(), None
    else:
        return ','.join(tmp).strip(), ' '.join(hrefs)

def remove_ref(text):
    for x in text.find_all('sup'):
        x.extract()
    return text

files = os.listdir('htmls/')

def sub_func(f_name):
    results = []
    with open('htmls/' + f_name, 'r') as f:
        soup = BeautifulSoup(f, 'html.parser')
        rs = soup.find_all(class_='wikitable sortable')
        
        for r in rs:
            heads = []
            rows = []
            for i, t_row in enumerate(r.find_all('tr')):
                if i == 0:
                    for h in t_row.find_all(['th', 'td']):
                        h = remove_ref(h)
                        if len(h.find_all('a')) > 0:
                            heads.append(process_link(h))
                        else:
                            heads.append((h.get_text().strip(), None))
                else:
                    row = []
                    for h in t_row.find_all(['th', 'td']):
                        h = remove_ref(h)
                        if len(h.find_all('a')) > 0:
                            row.append(process_link(h))
                        else:
                            row.append((h.get_text().strip(), None))
                    if all([len(cell[0]) == 0 for cell in row]):
                        continue
                    else:
                        rows.append(row)
            
            rows = rows[:20]
            if any([len(row) != len(heads) for row in rows]) or len(rows) < 8:
                continue
            else:
                text = r.previous_sibling
                while text is not None and text.name != 'p' and text.string is not None:
                    text = text.previous_sibling

                if text is None or text.string is None:
                    context = ''
                else:
                    context = text.string.strip()

                title = soup.title.string
                title = re.sub(' - Wikipedia', '', title)
                url = 'https://en.wikipedia.org/wiki/{}'.format('_'.join(title.split(' ')))
                results.append({'url': url, 'title': title, 'header': heads, 'data': rows, 'context': context})
    return results

rs = []
for f in files:
    tmp = sub_func(f)
    print(tmp)
    rs.append(tmp)

results = []
for r in rs:
    results = results + r

with open('processed_new_table.json', 'w') as f:
    json.dump(results, f, indent=2)

In [ ]:
import json

with open('processed_new_table.json', 'r') as f:
    tables = json.load(f)

#print("there are {} tables in total".format(len(tables)))

deletes = []
for i, table in enumerate(tables):
    # Remove sparse columns
    to_remove = []
    for j, h in enumerate(table['header']):
        #if j == 0:
        #    continue
        if 'Coordinates' in h[0][0] or 'Image' in h[0][0]:
            to_remove.append(j)
            continue
        
        count = 0
        total = len(table['data'])
        for d in table['data']:
            #print(d[j])
            if d[j][0][0] != '':
                count += 1
        
        if count / total < 0.5:
            to_remove.append(j)
    
    bias = 0
    for r in to_remove:
        del tables[i]['header'][r - bias]
        for _ in range(len(table['data'])):
            del tables[i]['data'][_][r - bias]
        bias += 1
    
    # Remove sparse rows
    to_remove = []
    for k in range(len(table['data'])):
        non_empty = [1 if _[0][0] != '' else 0 for _ in table['data'][k]]
        if sum(non_empty) < 0.5 * len(non_empty):
            to_remove.append(k)
    
    bias = 0
    for r in to_remove:        
        del tables[i]['data'][r - bias]
        bias += 1
    
    if len(table['header']) > 6:
        deletes.append(i)
    elif len(table['header']) <= 2:
        deletes.append(i)
    else:
        count = 0
        total = 0
        for row in table['data']:
            for cell in row:
                if len(cell[0][0]) != '':
                    if cell[1] == [None]:
                        count += 1                    
                    total += 1
        if count / total >= 0.7:
            deletes.append(i)

print('out of {} tables, {} need to be deleted'.format(len(tables), len(deletes)))

bias = 0
for i in deletes:
    del tables[i - bias]
    bias += 1

with open('processed_new_table_postfiltering.json', 'w') as f:
    json.dump(tables, f, indent=2)

In [ ]:
# coding: utf-8
import pandas
import json
from yattag import Doc
from yattag import indent
import random

with open('processed_new_table_postfiltering.json', 'r') as f:
    tables = json.load(f)

doc, tag, text = Doc().tagtext()

cache = ''

style = """
    th {
        padding-top: 12px;
        padding-bottom: 12px;
        text-align: left;
        background-color: #c9c9c9;
        color: black;
    }
    td, th {
        border: 1px solid #dddddd;
        text-align: left;
        padding: 8px;
    }
    td {
        padding-top: 12px;
        padding-bottom: 12px;
        text-align: left;
        background-color: #f0f0f0;
        color: black;
    }
    """

doc.asis('<!DOCTYPE html>')
with tag('html'):
    with tag('head'):
        with tag('style'):
            doc.asis(style)

        doc.asis('<meta charset=\"utf-8\">')
        doc.asis('<meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">')
        doc.asis('<title>Demonstration</title>')
        doc.asis('<link rel="icon" href="">') #Modifier ici pour le favicon
        doc.asis('<script defer src="https://use.fontawesome.com/releases/v5.3.1/js/all.js"></script>')
    
    
    with tag('body'):
        random.shuffle(tables)
        for table in tables[:200]:
            with tag('h3'):
                with tag('a', href=table['url']):
                    text(table['title'])
            
            with tag('h4'):
                text(table['context'])
                
            with tag('table', klass='wikitable', style="border:1"):
                with tag('tbody'):
                    with tag('tr'):
                        for cell in table['header']:
                            with tag('th'):
                                if cell[1] is not None:
                                    count = 0
                                    for t, s in zip(cell[0], cell[1]):
                                        if s is not None:
                                            with tag('a', href='http://edward.cs.ucsb.edu:6007/query?name=' + s):
                                                text(t)
                                            if count < len(cell[1]) - 1:
                                                text(', ')
                                        else:
                                            text(t)
                                            
                                        count += 1
                                else:
                                    text(cell[0])

                    for row in table['data']:
                        with tag('tr'):
                            for cell in row:
                                with tag('td'):
                                    if cell[1] is not None:
                                        count = 0
                                        for t, s in zip(cell[0], cell[1]):
                                            if s is not None:
                                                with tag('a', href='http://edward.cs.ucsb.edu:6007/query?name=' + s):
                                                    text(t)
                                                if count < len(cell[1]) - 1:
                                                    text(', ')
                                            else:
                                                text(t)
                                    else:
                                        text(cell[0])
            doc.stag('br')

result = doc.getvalue()

with open('index.html', 'w') as f:
    f.write(indent(result))

In [ ]:
import json
from bs4 import BeautifulSoup

def get_text(text):
    if 'Section::::' in text:
        text = text[:text.find('Section::::')]
    try:
        intro = text.split('\n\n')[1]
        d = BeautifulSoup(intro)
        intro = d.get_text().strip()
        return intro
    except Exception:
        return 'N/A'

dictionary = {}
with open('en.json') as f:
    for i, line in enumerate(f):
        d = json.loads(line.strip())
        page = '_'.join(d['title'].split(' '))
        dictionary[page] = get_text(d['text'])
        sys.stdout.write('finished {}/5989879 \r'.format(i))
        
with open('wiki-intro-with-ents-dict.json', 'w') as f:
    json.dump(dictionary, f)

In [ ]:
with open('processed_new_table_postfiltering.json', 'r') as f:
    tables = json.load(f)

dictionary = {}
missed = []

succ, fail = 0, 0
for table in tables:
    for row in table['data']:
        for cell in row:
            pages = cell[1]
            if pages is not None:
                for page in pages.split(' '):
                    page = page[6:].split('#')[0]
                    if page not in database:
                        fail += 1
                        print(page)
                        #database[page]
                    else:
                        succ += 1

sys.stdout.write('success/fail = {}/{} \r'.format(succ, fail))

In [ ]:
import urllib3
from bs4 import BeautifulSoup
import time
import re

http = urllib3.PoolManager()
urllib3.disable_warnings()

def get_summary(page):
    if page.startswith('https'):
        pass
    elif page.startswith('/wiki'):
        page = 'https://en.wikipedia.org{}'.format(page)
    else:
        page = 'https://en.wikipedia.org/wiki/{}'.format(page)
    
    r = http.request('GET', page)
    if r.status == 200:
        data = r.data.decode('utf-8')
        data = data.replace('</p><p>', ' ')        
        soup = BeautifulSoup(data, 'html.parser')

        div = soup.body.find("div", {"class": "mw-parser-output"})

        children = div.findChildren("p" , recursive=False)
        summary = 'N/A'
        for child in children:
            if child.get_text().strip() != "":
                html = str(child)
                html = html[html.index('>') + 1:].strip()
                if not html.startswith('<'):
                    summary = child.get_text().strip()
                    break
                elif html.startswith('<a>') or html.startswith('<b>') or \
                        html.startswith('<i>') or html.startswith('<a ') or html.startswith('<br>'):
                    summary = child.get_text().strip()
                    break
                else:
                    continue
        return summary
    elif r.status == 429:
        time.sleep(1)
        return get_summary(page)
    else:
        raise

get_summary('/wiki/Soe_Myint')

In [ ]:
import os

dictionary = {}
for f in os.listdir('hyperlinks/'):
    if f.endswith('json'):
        with open('hyperlinks/' + f, 'r') as fw:
            d = json.load(fw)
            dictionary.update(d)

print('totally {}'.format(len(dictionary)))

failed = [k for k, v in dictionary.items() if v == 'N/A']
print('failed {} items'.format(len(failed)))

with open('wikipedia/round1.json', 'w') as f:
    json.dump(dictionary, f, indent=2)
    
with open('wikipedia/round1_failed.json', 'w') as f:
    json.dump(failed, f, indent=2)

In [ ]:
with open('wikipedia/round1.json') as f:
    dictionary = json.load(f)
    
with open('wikipedia/round2.json') as f:
    dictionary.update(json.load(f))

In [ ]:
#print(len(dictionary))
import re
import json
import urllib.parse

#for k, v in dictionary.items():
#    dictionary[k] = re.sub(r'\[[\d]+\]', '', v).strip()
with open('wikipedia/merged.json') as f:
    dictionary = json.load(f)

merged_unquote = {}
for k, v in dictionary.items():
    merged_unquote[urllib.parse.unquote(k)] = v

with open('wikipedia/merged_unquote.json', 'w') as f:
    json.dump(merged_unquote, f, indent=2)

In [ ]:
import json

with open('processed_new_table_postfiltering.json', 'r') as f:
    data = json.load(f)

for i, d in enumerate(data):
    d['idx'] = i
    with open('tables/{}.json'.format(i), 'w') as f:
        json.dump(d, f, indent=2)

# Code for Generating the request data

In [ ]:
import os
import json
import sys
from utils import *
import re
import copy
from shutil import copyfile

def recover(string):
    string = string[6:]
    string = string.replace('_', ' ')
    return string
    
def clean_text(k, string):
    if "Initial visibility" in string:
        return recover(k)
    
    position = string.find("mw-parser-output")
    if position != -1:
        left_quote = position - 1
        while left_quote >= 0 and string[left_quote] != '(':
            left_quote -= 1
        right_quote = position + 1
        while right_quote < len(string) and string[right_quote] != ')':
            right_quote += 1
        
        string = string[:left_quote] + " " + string[right_quote + 1:]
        
        position = string.find("mw-parser-output")
        if position != -1:
            #print(string)
            right_quote = position + 1
            while right_quote < len(string) and string[right_quote] != '\n':
                right_quote += 1
            #print("----------------")
            string = string[:position] + string[right_quote + 1:]
            #print(string)
            #print("################")
    
    string = string.replace(u'\xa0', u' ')
    string = string.replace('\ufeff', '')
    string = string.replace(u'\u200e', u' ')
    string = string.replace('–', '-')
    string = string.replace(u'\u2009', u' ')
    string = string.replace(u'\u2010', u' - ')
    string = string.replace(u'\u2011', u' - ')
    string = string.replace(u'\u2012', u' - ')
    string = string.replace(u'\u2013', u' - ')
    string = string.replace(u'\u2014', u' - ')
    string = string.replace(u'\u2015', u' - ')
    string = string.replace(u'\u2018', u'')
    string = string.replace(u'\u2019', u'')
    string = string.replace(u'\u201c', u'')
    string = string.replace(u'\u201d', u'')    
    
    string = string.replace(u'"', u'')
    string = re.sub(r'[\n]+', '\n', string)
    
    string = re.sub(r'\.+', '.', string)
    string = re.sub(r' +', ' ', string)
    
    #string = re.sub(r"'+", "'", string)
    #string = string.replace(" '", " ")
    #string = string.replace("' ", " ")
    string = filter_firstKsents(string, 12)
    
    return string

with open('wikipedia/merged_unquote.json', 'r') as f:
    merged_unquote = json.load(f)

for k in merged_unquote:
    merged_unquote[k] = clean_text(k, merged_unquote[k])

def func(f_id):
    if f_id.endswith('.json'):
        with open('tables/' + f_id) as f:
            table = json.load(f)
    
    local_dict = {}
    for d in table['header']:
        for url in d[1]:
            if url:
                url = urllib.parse.unquote(url)
                local_dict[url] = merged_unquote[url]
    
    for row in table['data']:
        for cell in row:
            for url in cell[1]:
                if url:
                    url = urllib.parse.unquote(url)
                    local_dict[url] = merged_unquote[url]
    #count += 1
    #sys.stdout.write("finished {} tables \r".format(count))
    with open('request_wo_filter/{}'.format(f_id), 'w') as f:
        json.dump(local_dict, f, indent=2)

from multiprocessing import Pool

pool = Pool(64)
results_func = pool.map(func, os.listdir('tables/'))

pool.close()
pool.join()

copyfile('request/example.json', 'request_wo_filter/examples.json')
copyfile('request/example_numeric.json', 'request_wo_filter/example_numeric.json')

In [ ]:
import json
from transformers import *
import torch

device = torch.device('cuda:5')

pretrained_weights = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

model.to(device)

with open('Mixed-Reasoning/collected_data.json') as f:
    data = json.load(f)

for d in data:
    table_id = d[0]
    with open('request/{}.json'.format(table_id)) as f:
        requested_documents = json.load(f)
    
    idx2key = []
    tmp = []
    for k, v in requested_documents.items():
        tokenized_paragraph = tokenizer.tokenize(v)
        if len(tokenized_paragraph) < 512:
            tokenized_paragraph = tokenized_paragraph[:512]
        
        idxs = tokenizer.convert_tokens_to_ids(tokenized_paragraph)
        tensor = torch.LongTensor(idxs).unsqueeze(0).to(device)
        _, r2 = model(tensor)
        tmp.append(r2)
        idx2key.append(k)
    
    requested_repr = torch.cat(tmp, 0)
    for q, a in d[1:]:
        idxs = tokenizer.encode(q)
        print(idxs)
        tensor = torch.LongTensor(idxs).unsqueeze(0).to(device)
        _, r2 = model(tensor)
        r2 = r2.repeat(requested_repr.shape[0], 1)
        
        similarity = torch.nn.functional.cosine_similarity(r2, requested_repr, dim=1)
        print(similarity)
    
    break

# Step-1 Preprocessing

In [ ]:
import json
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
import nltk.data
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
from utils import *
import re
import random
import nltk
import dateparser
from dateparser.search import search_dates
from dateparser import parse
import json
from multiprocessing import Pool
import os

stopWords = set(stopwords.words('english'))
tfidf = TfidfVectorizer(strip_accents="unicode", ngram_range=(2, 3), stop_words=stopWords)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    
def longestSubstringFinder(S,T):
    S = S.lower()
    T = T.lower()
    m = len(S)
    n = len(T)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_set = set()
    for i in range(m):
        for j in range(n):
            if S[i] == T[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_set = set()
                    longest = c
                    lcs_set.add(S[i-c+1:i+1])
                elif c == longest:
                    lcs_set.add(S[i-c+1:i+1])
    
    return longest, lcs_set

def longest_match_distance(str1s, str2s):
    longest_string = []
    for str1 in str1s:
        longest_string.append([])
        for str2 in str2s:
            length, _ = longestSubstringFinder(str1, str2)
            longest_string[-1].append(1 - length / len(str1))
    return longest_string

def searchForAnswer(answer, table, passages, mapping_entity):
    results = []
    correction = None
    for i, row in enumerate(table['data']):
        for j, cell in enumerate(row):
            success = False
            for content, url in zip(cell[0], cell[1]):
                if answer.lower() == content.lower():
                    results.append((content, (i, j), url, 'table'))
                    success = True
                    break
                elif " " + answer.lower() + " " in " " + content.lower() + " ":
                    correction = content
                    results.append((content, (i, j), url, 'table'))
                else:
                    pass
            
            if not success and len(cell[0]) > 1:
                content = ' , '.join(cell[0])
                if answer == content:
                    results.append((content, (i, j), None, 'table'))
                elif len(answer) > 3 and " " + answer.lower() + " " in " " + content.lower() + " ":
                    correction = content
                    results.append((content, (i, j), None, 'table'))

    if len(results) > 0:
        return correction, results
        
    for k, v in passages.items():
        if " " + answer.lower() + " " in " " + v.lower() + " ":
            for content, locs in mapping_entity[k].items():
                for loc in locs:
                    results.append((content, loc, k, 'passage'))
    
    return None, results

def searchForAnswerWithoutSpace(answer, passages, mapping_entity):
    correction = None
    results = []
    for k, v in passages.items():
        tmp = (" " + v.lower()).find(" " + answer.lower())
        if tmp != -1:
            length = len(answer)
            while tmp + length < len(v) and v[tmp + length] != " ":
                length += 1
            correction = v[tmp:tmp + length]
            for content, locs in mapping_entity[k].items():
                for loc in locs:
                    results.append((content, loc, k, 'passage'))
            break

    return correction, results

def get_edit_distance_equal_1(answer, table):
    results = []
    for i, row in enumerate(table['data']):
        for j, cell in enumerate(row):
            for tmp, url in zip(cell[0], cell[1]):
                dist = nltk.edit_distance(answer, tmp)
                if dist == 1:
                    results.append((tmp, (i, j), url, 'table'))
    return results

def fixing_answer(string):
    if ',' in string:
        tmp = string.split(',')[0].strip()
        if not tmp.isdigit():
            string = [tmp]
        else:
            return None
    elif ' and ' in string:
        string = [_.strip() for _ in string.split(' and ')]
    elif '(' and ')' in string:
        tmp = re.sub(r'([^\(\)]+) \((.+)\)$', r'\1###\2', string)
        string = [_.strip() for _ in tmp.split('###')]
    elif '-' in string:
        if ' - ' in string:
            tmp = string.replace(' - ', '-')
        else:
            tmp = string.replace('-', ' - ')
        string = [tmp, string.split('-')[0].strip(), string.split('-')[1].strip()]
    elif string.startswith('#'):
        string = [string.lstrip('#').strip()]
    else:
        return None
    
    return string
        
def step1(d):
    results = []
    table_id = d[0]
    #if table_id != 12040:
    #    return []

    with open('request_tok/{}.json'.format(table_id)) as f:
        requested_documents = json.load(f)
    with open('tables_tok/{}.json'.format(table_id)) as f:
        table = json.load(f)    
    
    #title_wiki = table['url'][len('https://en.wikipedia.org'):]
    #del requested_documents[title_wiki]
    # Finding the answer and links to table
    qs = []
    ans = []
    links = []
    
    # Mapping entity link to cell, entity link to surface word
    #mapping_entity_loc = {}
    mapping_entity = {}
    for row_idx, row in enumerate(table['data']):
        for col_idx, cell in enumerate(row):
            for i, ent in enumerate(cell[1]):
                if ent:
                    if ent not in mapping_entity:
                        mapping_entity[ent] = {cell[0][i]: [(row_idx, col_idx)]}
                    else:
                        if cell[0][i] not in mapping_entity[ent]:
                            mapping_entity[ent][cell[0][i]] = [(row_idx, col_idx)]
                        else:
                            mapping_entity[ent][cell[0][i]] = mapping_entity[ent][cell[0][i]] + [(row_idx, col_idx)]
    
    for col_idx, header in enumerate(table['header']):
        for i, ent in enumerate(header[1]):
            if ent:
                if ent not in mapping_entity:
                    mapping_entity[ent] = {header[0][i]: [(-1, col_idx)]}
                else:
                    if header[0][i] not in mapping_entity[ent]:
                        mapping_entity[ent][header[0][i]] = [(-1, col_idx)]
                    else:
                        mapping_entity[ent][header[0][i]] = mapping_entity[ent][header[0][i]] + [(-1, col_idx)]
    
    debug = False
    # loop through the qa pairs
    for q, a in d[1:]:
        correction, tmp = searchForAnswer(a, table, requested_documents, mapping_entity)
        if len(tmp) == 0 and len(a) >= 3 and not a.isdigit():
            # See if the space becomes a problem
            correction, tmp = searchForAnswerWithoutSpace(a, requested_documents, mapping_entity)
            if len(tmp) > 0:
                if debug:
                    print("correct span! {} -> {}".format(a, correction))
                pass
            else:
                # correct the spelling                
                tmp = get_edit_distance_equal_1(a, table)
                if len(tmp) > 0:
                    if debug:
                        print("correct spelling! {} -> {}".format(a, tmp[0][0]))
                    correction = tmp[0][0]
                else:
                    # Split the answer
                    fixed_as = fixing_answer(a)
                    if fixed_as:
                        for correction in fixed_as:
                            _, tmp = searchForAnswer(correction, table, requested_documents, mapping_entity)
                            if len(tmp) > 0:
                                if debug:
                                    print("correct splitting! {} -> {}".format(a, correction))
                                break
            
            if len(tmp) > 4:
                if debug:
                    print("many uncertainties for {}, decide not to replace it".format(a))
                tmp = []
            elif correction and len(tmp) > 0:
                a = correction.strip()
            else:
                pass
            
        elif correction:
            if len(correction) < 2 * len(a) : 
                if debug:
                    print("cell correction! {} -> {}".format(a, correction))
                a = correction.strip()
        
        ans.append((a, tmp))
        qs.append(q)
        

    keys = []
    paras = []
    for k, v in requested_documents.items():
        for _ in tokenizer.tokenize(v):
            keys.append(k)
            paras.append(_)
    
    try:
        para_feature = tfidf.fit_transform(paras)
        q_feature = tfidf.transform(qs)
    except Exception:
        print("failed on table {}".format(table_id))
        return []
        
    dist_match = longest_match_distance(qs, paras)
    dist = pairwise_distances(q_feature, para_feature, 'cosine')
    
    threshold = 0.95
    best_threshold = 0.70
    for i in range(len(qs)):
        tfidf_nodes = []
        string_nodes = []

        min_dist = {}
        tfidf_best_match = ('N/A', None, 1.)
        for k, para, d in zip(keys, paras, dist[i]):
            if d < min_dist.get(k, threshold):
                min_dist[k] = d
                if d < tfidf_best_match[-1]:
                    tfidf_best_match = (k, para, d)
                if d <= best_threshold:
                    for content, locs in mapping_entity[k].items():
                        for loc in locs:
                            tfidf_nodes.append((content, loc, k, para, d))
        
        if tfidf_best_match[0] != 'N/A':
            if tfidf_best_match[-1] > best_threshold:
                for content, locs in mapping_entity[k].items():
                    for loc in locs:
                        tfidf_nodes.append((content, loc, k, tfidf_best_match[1], tfidf_best_match[2]))

        min_dist = {}
        string_best_match = ('N/A', None, 1.)
        for k, para, d in zip(keys, paras, dist_match[i]):
            if d < min_dist.get(k, threshold):
                min_dist[k] = d
                if d < string_best_match[-1]:
                    string_best_match = (k, para, d)
                if d <= best_threshold:
                    for content, locs in mapping_entity[k].items():
                        for loc in locs:
                            string_nodes.append((content, loc, k, para, d))
                    
        if string_best_match[0] != 'N/A':
            if string_best_match[-1] > best_threshold:
                for content, locs in mapping_entity[k].items():
                    for loc in locs:
                        string_nodes.append((content, loc, k, string_best_match[1], string_best_match[2]))
        
        results.append({'table_id': table_id, 'question': qs[i], 'answer-text': ans[i][0], 
                        'answer-node': ans[i][1], 'tf-idf': tfidf_nodes, 
                        'string-overlap': string_nodes})
    
    return results

def convert2num(string):
    string = string.replace(',', '')
    if string.endswith('%'):
        string = string.rstrip('%')
    try:
        string = float(string)
        return string
    except Exception:
        return None
    
def find_superlative(table_id, table):
    if not os.path.exists('tables_tmp/{}.json'.format(table_id)):
        mapping = {}
        headers = [_[0][0] for _ in table['header']]
        for j in range(len(table['header'])):
            mapping[headers[j]] = []
            activate_date_or_num = None
            if headers[j] not in ['#', 'Type', 'Name', 'Location', 'Position', 'Category', 'Nationality',
                                  'School', 'Notes', 'Notability', 'Country']:
                for i, row in enumerate(table['data']):
                    if len(table['data'][i][j][0]) > 1:
                        continue

                    data = table['data'][i][j][0][0]
                    if data in ['', '-']:
                        continue

                    num = convert2num(data)
                    if num and data.isdigit() and num > 1000 and num < 2020 and activate_date_or_num in ['date', None]:
                        date_format = parse(data)
                        mapping[headers[j]].append((date_format, 'date', [data, (i, j), None, None, 1.0]))
                        activate_date_or_num = 'date'
                    elif num and activate_date_or_num in ['num', None]: 
                        mapping[headers[j]].append((num, 'number', [data, (i, j), None, None, 1.0]))
                        activate_date_or_num = 'num'
                    else:
                        try:
                            date_format = parse(data)
                            if date_format and activate_date_or_num in ['date', None]:
                                mapping[headers[j]].append((date_format, 'date', [data, (i, j), None, None, 1.0]))
                                activate_date_or_num = 'date'
                        except Exception:
                            continue

            if len(mapping[headers[j]]) < 0.3 * len(table['data']):
                mapping[headers[j]] = []

        nodes = []
        for k, v in mapping.items():
            if len(v) > 0:
                tmp = sorted(v, key = lambda x: x[0])
                if tmp[0][1] == 'number':
                    tmp_node = tmp[0][-1]
                    tmp_node[3] = 'minimum'
                    nodes.append(tmp_node)
                    tmp_node = tmp[-1][-1]
                    tmp_node[3] = 'maximum'
                    nodes.append(tmp_node)
                else:
                    tmp_node = tmp[0][-1]
                    tmp_node[3] = 'earliest'
                    nodes.append(tmp_node)
                    tmp_node = tmp[-1][-1]
                    tmp_node[3] = 'latest'
                    nodes.append(tmp_node)

        with open('tables_tmp/{}.json'.format(table_id), 'w') as f:
            json.dump(nodes, f)
    else:
        with open('tables_tmp/{}.json'.format(table_id), 'r') as f:
            nodes = json.load(f)

    return nodes

def step2(d):
    threshold = 90
    # LINKING THE CELL DATA
    triggers = ['JJR', 'JJS', 'RBR', 'RBS']
    
    new_processed = []

    table_id = d['table_id']
    with open('tables_tok/{}.json'.format(table_id)) as f:
        table = json.load(f)        

    tmp_link = []
    for row_idx, row in enumerate(table['data']):
        for col_idx, cell in enumerate(row):
            if cell[0] != ['']:
                for ent in cell[0]:
                    ratio = fuzz.partial_ratio(' ' + ent.lower() + ' ', ' ' + d['question'].lower() + ' ')
                    if ratio > threshold:
                        tmp_link.append((ent, (row_idx, col_idx), None, 'string match', ratio / 100))

    pos_tags = " ".join([_[1] for _ in nltk.pos_tag(d['question'].split(' '))])
    d['question_postag'] = pos_tags

    d['links'] = tmp_link
    if any([_ in pos_tags for _ in triggers]):
        #try:
        tmp = find_superlative(table_id, table)
        d['links'] = d['links'] + tmp
        #except Exception:
        #    print("failed with table {}".format(table_id))

    return d


def hash_string(string):
    import hashlib
    sha = hashlib.sha256()
    sha.update(string.encode())
    return sha.hexdigest()[:16]


def step3(processed):
    trivial, easy, medium, hard, no_answer, number, yesorno, repeated = 0, 0, 0, 0, 0, 0, 0, 0
    from_passage, from_cell, from_calculation = 0, 0, 0
    new_processed = []
    used_question_id = set([])
    
    question_type = ''
    where_from = ''
    for p in processed:
        p['question_id'] = hash_string(p['question'])
        
        if p['question_id'] in used_question_id:
            repeated +=1
            continue
        elif p['answer-text'].lower() in ['yes', 'no']:
            yesorno += 1
            continue
        elif len(p['answer-text'].split(' ')) > 15:
            yesorno += 1
            continue
        else:
            if len(p['answer-node']) > 1 and p['answer-node'][0][-1] == 'table':
                with open('tables_tok/{}.json'.format(p['table_id']), 'r') as f:
                    table = json.load(f)
                headers = [" , ".join(cell[0]) for cell in table['header']]
                potential_headers = set()
                for h in headers:
                    if " " + h.lower() + " " in " " + p['question'].lower() + " ":
                        potential_headers.add(h)

                if len(potential_headers) > 0:
                    remaining_nodes = []
                    for n in p['answer-node']:
                        if headers[n[1][1]] in potential_headers:
                            remaining_nodes.append(n)

                    if len(remaining_nodes) > 0 and len(remaining_nodes) < len(p['answer-node']):
                        p['answer-node'] = remaining_nodes

            # Categorize the difficulty level
            number_trigger = ['how many', 'how much', 'how long', 'how far', 'how old', 'difference', 'total']
            answer_node = p['answer-node']
            if len(answer_node) == 0:
                if any([x in p['question'].lower() for x in number_trigger]):
                    number += 1
                    question_type = 'numeric'
                else:
                    no_answer += 1
                    continue
            else:
                matching_cells = []
                if p['tf-idf']:
                    matching_cells.extend([tuple(_[1]) for _ in p['tf-idf']])
                if p['string-overlap']:
                    matching_cells.extend([tuple(_[1]) for _ in p['string-overlap']])
                linking_cells = [tuple(_[1]) for _ in p['links']]
                matching_cells = set(matching_cells)
                linking_cells = set(linking_cells)

                evidence_cells = matching_cells | linking_cells
                answer_row = set([_[1][0] for _ in answer_node])
                evidence_row = set([_[0] for _ in evidence_cells])
                intersect_row = answer_row & evidence_row

                if len(intersect_row) > 0:
                    new_answer_nodes = []
                    for node in p['answer-node']:
                        if node[1][0] in intersect_row:
                            new_answer_nodes.append(node)
                    p['answer-node'] = new_answer_nodes
                
                answer_cells = set([tuple(_[1]) for _ in p['answer-node']])
                if len(evidence_cells & answer_cells) > 0:
                    new_answer_nodes = []
                    for node in p['answer-node']:
                        if tuple(node[1]) in evidence_cells:
                            new_answer_nodes.append(node)
                    p['answer-node'] = new_answer_nodes
                
                answer_cells = set([tuple(_[1]) for _ in p['answer-node']])
                
                if matching_cells == answer_cells:
                    trivial += 1
                    question_type = 'trivial'
                elif len(evidence_cells & answer_cells) > 0:
                    easy += 1
                    question_type = 'easy'
                else:
                    if len(intersect_row) > 0:
                        medium += 1
                        question_type = 'medium'
                    else:
                        hard += 1
                        question_type = 'hard'

        p['type'] = question_type

        if p['type'] != 'trivial':
            if len(answer_node) > 0:
                if answer_node[0][-1] == 'passage':
                    from_passage += 1
                else:
                    from_cell += 1
                p['where'] = answer_node[0][-1]
            else:
                from_calculation += 1
                p['where'] = 'calculation'
                
            new_processed.append(p)
            used_question_id.add(p['question_id'])

    print("trivial: {}, easy: {}, medium: {}, hard: {}, number: {}, no answer: {}, yes/no: {}, repeated: {}".
          format(trivial, easy, medium, hard, number, no_answer, yesorno, repeated))
    print("from cell: {}, from passage: {}, from_calculation: {}".format(from_cell, from_passage, from_calculation))

    return new_processed

In [ ]:
with open('Mixed-Reasoning/collected_data.json') as f:
    data = json.load(f)
print("running data for {} tables".format(len(data)))

from multiprocessing import Pool

pool = Pool(64)
results_func = pool.map(step1, data)

pool.close()
pool.join()

results = []
for _ in results_func:
    results.extend(_)

#with open('Mixed-Reasoning/processed_step1.json', 'w') as f:
#    json.dump(results, f, indent=2)
    
#with open('Mixed-Reasoning/processed_step1.json', 'r') as f:
#    processed = json.load(f)

pool = Pool(64)
    
results = pool.map(step2, results)

pool.close()
pool.join()    
#with open('Mixed-Reasoning/processed_step2.json', 'r') as f:
#    processed = json.load(f)
with open('Mixed-Reasoning/processed_step2.json', 'w') as f:
    json.dump(results, f, indent=2)

In [ ]:
with open('Mixed-Reasoning/collected_data.json') as f:
    data = json.load(f)
print("running data for {} tables".format(len(data)))

from multiprocessing import Pool

pool = Pool(64)
results_func = pool.map(step1, data)

pool.close()
pool.join()

results = []
for _ in results_func:
    results.extend(_)



In [ ]:
print("before filtering, there are {} instances".format(len(results)))
processed = step3(results)
processed = sorted(processed, key=lambda x: x['question_id'])

#with open('Mixed-Reasoning/processed_step3_indent2.json', 'w') as f:
#    json.dump(processed, f, indent=2)
with open('Mixed-Reasoning/processed_step3.json', 'w') as f:
    json.dump(processed, f)

num_slots = 0
for d in processed:
    num_slots += len(d['tf-idf']) + len(d['string-overlap']) + len(d['links'])

print("average link is {} for {} instances".format(num_slots / len(processed), len(processed)))

In [ ]:
with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    new_processed = json.load(f)

with open('Mixed-Reasoning/dev_ids.json') as f:
    dev_ids = json.load(f)

medium_split = []
for d in new_processed:
    if d['type'] == 'medium':
        tmp = set()
        for n in d['answer-node']:
            tmp.add((n[0], n[2]))
        if len(tmp) == 1:
            #with open('request_tok/{}.json'.format(d['table_id'])) as f:
            #    request_document = json.load(f)
            #document = request_document[list(tmp)[0][1]]
            #if len(document.split(' ')) > 100:
            if d['question_id'] in dev_ids:
                medium_split.append({'table_id': d['table_id'], 'question_id': d['question_id'],
                                    'answer-text': d['answer-text'], 'answer-node': d['answer-node'][0],
                                    'question': d['question']})
            #else:
            #    pass
print("The medium split has {} entries".format(len(medium_split)))

with open('Mixed-Reasoning/dev_processed_step3_medium.json', 'w') as f:
    json.dump(medium_split, f, indent=2)

In [ ]:
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
from random import *
import math
import seaborn as sns
import copy

n = numpy.zeros((20, 6))

with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    data = json.load(f)
    
for d in data:
    table_id = d['table_id']
    with open('tables/{}.json'.format(table_id)) as f:
        table = json.load(f)
    columns = len(table['header'])
    rows = len(table['data'])
    
    for node in d['answer-node']:
        i = round((node[1][0] + 1) / rows * 20) - 1
        j = round((node[1][1] + 1) / columns * 6) - 1
        if n[i, j] < 1200:
            n[i, j] += 1

new_n = copy.copy(n)
for i in range(0, 5):
    new_n[:, i] = (new_n[:, i] + new_n[:, i + 1]) / 2
    
sns.heatmap(new_n, cmap="YlGnBu", xticklabels=False, yticklabels=False)
plt.savefig('table-distribution.jpg', dpi=1000)

In [ ]:
with open('Mixed-Reasoning/stage3_training_data.json') as f:
    data = json.load(f)

from collections import Counter
import math

total = 484
grid_size = int(math.sqrt(total))

counter = Counter()
for d in data:
    start_position = d['answers'][0]['answer_start']
    if start_position != 8:
        counter.update([start_position])
        
heat = [0 for _ in range(0, total)]
for c, n in counter.most_common():
    if c < total:
        heat[c] = min(n, 100)

n = numpy.array(heat).reshape([grid_size, grid_size])

sns.heatmap(n, cmap="YlGnBu", xticklabels=False, yticklabels=False)
plt.savefig('passage-distribution.jpg', dpi=1000)

# Cleaning the table

In [ ]:
import os

for fn in os.listdir('tables/'):
    if fn.endswith('.json'):
        with open('tables/{}'.format(fn)) as f:
            table = json.load(f)
        
        headers = table['header']
        if headers[0][0] == ['']:
            for i in range(len(table['data'])):
                del table['data'][i][0]
        
            del headers[0]

            with open('tables/{}'.format(fn), 'w') as f:
                json.dump(table, f, indent=2)

In [ ]:
for fn in os.listdir('tables/'):
    if fn.endswith('.json'):
        with open('tables/{}'.format(fn)) as f:
            table = json.load(f)

        headers = table['header']

        if any([_[0] == ['Rank'] for _ in headers]):
            if table['data'][0][0][0] == ['']:
                for i in range(len(table['data'])):
                    if table['data'][i][0][0] == ['']:
                        table['data'][i][0][0] = [str(i + 1)]
                
                with open('tables/{}'.format(fn), 'w') as f:
                    json.dump(table, f, indent=2)
        
        if any([_[0] == ['Place'] for _ in headers]):
            if table['data'][0][0][0] == ['']:
                for i in range(len(table['data'])):
                    if table['data'][i][0][0] == ['']:
                        table['data'][i][0][0] = [str(i + 1)]
                
                with open('tables/{}'.format(fn), 'w') as f:
                    json.dump(table, f, indent=2)

In [ ]:
import json
import os
import re

def clean_cell_text(string):
    string = string.rstrip('^')
    string = string.replace('\u200e', '')
    string = string.replace('\ufeff', '')
    string = string.replace('–', '-')
    string = string.replace(u'\u2009', u' ')
    string = string.replace(u'\u2010', u' - ')
    string = string.replace(u'\u2011', u' - ')
    string = string.replace(u'\u2012', u' - ')
    string = string.replace(u'\u2013', u' - ')
    string = string.replace(u'\u2014', u' - ')
    string = string.replace(u'\u2015', u' - ')
    string = string.replace(u'\u2018', u'')
    string = string.replace(u'\u2019', u'')
    string = string.replace(u'\u201c', u'')
    string = string.replace(u'\u201d', u'')
    string = re.sub(r' +', ' ', string)
    string = string.strip()
    return string
    
for fn in os.listdir('tables/'):
    with open('tables/{}'.format(fn)) as f:
        table = json.load(f)
    
    for row_idx, row in enumerate(table['data']):
        for col_idx, cell in enumerate(row):
            for i, ent in enumerate(cell[0]):
                if ent:
                    table['data'][row_idx][col_idx][0][i] = clean_cell_text(ent)
    
    for col_idx, header in enumerate(table['header']):
        for i, ent in enumerate(header[0]):
            if ent:
                table['header'][col_idx][0][i] = clean_cell_text(ent)
    
    with open('tables/{}'.format(fn), 'w') as f:
        json.dump(table, f, indent=2)

# Generating dev and test data

In [ ]:
import random
import json
from utils import url2dockey, filter_firstKsents

with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    data = json.load(f)

#train_keys = []
#dev_keys = []
medium_keys = []
hard_keys = []
for k in data:
    if k['type'] in ['medium', 'numeric']:
        medium_keys.append(k['question_id'])
    elif k['type'] in ['hard']:
        hard_keys.append(k['question_id'])

random.shuffle(medium_keys)
random.shuffle(hard_keys)

size = 8000
dev_keys = medium_keys[:size] + hard_keys[:1000]
test_keys = medium_keys[size:size * 2] + hard_keys[1000:2000]


with open('Mixed-Reasoning/dev_ids.json', 'w') as f:
    json.dump(dev_keys, f)
    
with open('Mixed-Reasoning/test_ids.json', 'w') as f:
    json.dump(test_keys, f)

with open('Mixed-Reasoning/medium_dev_ids.json', 'w') as f:
    json.dump(medium_keys[:size], f)

with open('Mixed-Reasoning/hard_dev_ids.json', 'w') as f:
    json.dump(hard_keys[:1000], f)

"""
dev_size = int(len(data) * 0.1)
random.shuffle(potential_dev_keys)

dev_keys = potential_dev_keys[:int(dev_size//2)]
test_keys = potential_dev_keys[int(dev_size//2) : dev_size]

train_keys = train_keys + potential_dev_keys[dev_size:]

"""

In [ ]:
for opt in ['dev', 'test']:
    with open('Mixed-Reasoning/code/predictions.{}.json'.format(opt), 'r') as f:
        results = json.load(f)

    wrong_question_ids = []
    correct_question_ids = []
    for r in results:
        if r['answer-text'] != r['pred']:
            wrong_question_ids.append(r['question_id'])
        else:
            correct_question_ids.append(r['question_id'])

    random.shuffle(wrong_question_ids)
    random.shuffle(correct_question_ids)

    wrong_question_ids = wrong_question_ids[:2000]
    correct_question_ids = correct_question_ids[:1500]

    dev_ids = wrong_question_ids + correct_question_ids
    random.shuffle(dev_ids)

    with open('Mixed-Reasoning/filtered_{}_ids.json'.format(opt), 'w') as f:
        json.dump(dev_ids, f)

# Generating the released dataset

In [ ]:
import json
import os

with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    data = json.load(f)
with open('Mixed-Reasoning/filtered_dev_ids.json', 'r') as f:
    dev_keys = set(json.load(f))
with open('Mixed-Reasoning/filtered_test_ids.json', 'r') as f:
    test_keys = set(json.load(f))

if not os.path.exists('Mixed-Reasoning/released_data'):
    os.mkdir('Mixed-Reasoning/released_data')

train_data = []
dev_data = []
test_data = []

dev_reference = {'reference': {}, 'passage':[], 'table': []}
test_reference = {'reference': {}, 'passage':[], 'table': []}

for d in data:
    if d['question_id'] in dev_keys:
        del d['tf-idf']
        del d['string-overlap']
        del d['links']
        del d['type']
        dev_data.append(d)
        dev_reference['reference'][d['question_id']] = d['answer-text']
        if d['where'] == 'table':
            dev_reference['table'].append(d['question_id'])
        else:
            dev_reference['passage'].append(d['question_id'])
        
    elif d['question_id'] in test_keys:
        del d['tf-idf']
        del d['string-overlap']
        del d['links']
        del d['type']
        test_data.append(d)
        test_reference['reference'][d['question_id']] = d['answer-text']
        if d['where'] == 'table':
            test_reference['table'].append(d['question_id'])
        else:
            test_reference['passage'].append(d['question_id'])
        del d['answer-text']
        del d['answer-node']
        del d['where']    
    else:
        del d['tf-idf']
        del d['string-overlap']
        del d['links']
        del d['type']
        train_data.append(d)

with open('Mixed-Reasoning/released_data/train.json', 'w') as f:
    json.dump(train_data, f, indent=2)

with open('Mixed-Reasoning/released_data/dev.json', 'w') as f:
    json.dump(dev_data, f, indent=2)

with open('Mixed-Reasoning/released_data/test.json', 'w') as f:
    json.dump(test_data, f, indent=2)

with open('Mixed-Reasoning/released_data/dev_reference.json', 'w') as f:
    json.dump(dev_reference, f)

with open('Mixed-Reasoning/released_data/test_reference.json', 'w') as f:
    json.dump(test_reference, f)
    
print("there are {} training, {} dev, {} test instances".format(len(train_data), len(dev_data), len(test_data)))

# Generating Test/Dev Inputs

In [ ]:
with open('Mixed-Reasoning/dev_ids.json', 'r') as f:
    dev_keys = set(json.load(f))
with open('Mixed-Reasoning/test_ids.json', 'r') as f:
    test_keys = set(json.load(f))
with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    data = json.load(f)
    
dev_split = []

for d in data:
    if d['question_id'] in dev_keys:
        table_id = d['table_id']
        with open('tables_tok/{}.json'.format(table_id), 'r') as f:
            table = json.load(f)
        headers = [cell[0][0] for cell in table['header']]

        answer_nodes = d['answer-node']
        answer_rows = set([_[1][0] for _ in answer_nodes])

        tmp = []
        labels = []
        for node in d['tf-idf']:
            tmp.append(node + [headers[node[1][1]], 'tf-idf'])
        for node in d['string-overlap']:
            tmp.append(node + [headers[node[1][1]], 'string-overlap'])            
        for node in d['links']:
            tmp.append(node + [headers[node[1][1]], 'links'])
        dev_split.append({'question': d['question'], 'question_id': d['question_id'], 'table_id': d['table_id'], 
                          'nodes': tmp, 'answer-text': d['answer-text']})
        
with open('Mixed-Reasoning/dev_input.json', 'w') as f:
    json.dump(dev_split, f, indent=2)
    
test_split = []

for d in data:
    if d['question_id'] in test_keys:
        table_id = d['table_id']
        with open('tables_tok/{}.json'.format(table_id), 'r') as f:
            table = json.load(f)
        headers = [cell[0][0] for cell in table['header']]

        answer_nodes = d['answer-node']
        answer_rows = set([_[1][0] for _ in answer_nodes])

        tmp = []
        labels = []
        for node in d['tf-idf']:
            tmp.append(node + [headers[node[1][1]], 'tf-idf'])
        for node in d['string-overlap']:
            tmp.append(node + [headers[node[1][1]], 'string-overlap'])            
        for node in d['links']:
            tmp.append(node + [headers[node[1][1]], 'links'])
        
        test_split.append({'question': d['question'], 'question_id': d['question_id'], 'table_id': d['table_id'], 
                          'nodes': tmp, 'answer-text': d['answer-text']})
        
with open('Mixed-Reasoning/test_input.json', 'w') as f:
    json.dump(test_split, f, indent=2)

# Creating the stage1/2/3 training data

In [ ]:
with open('Mixed-Reasoning/dev_ids.json', 'r') as f:
    dev_keys = json.load(f)
    
with open('Mixed-Reasoning/test_ids.json', 'r') as f:
    test_keys = json.load(f)

with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    data = json.load(f)


with open('Mixed-Reasoning/dev_ids.json', 'r') as f:
    dev_keys = set(json.load(f))

train_split = []
dev_split = []

for d in data:
    if d['type'] in ['medium', 'easy']:
        table_id = d['table_id']
        with open('tables_tok/{}.json'.format(table_id), 'r') as f:
            table = json.load(f)
        headers = [" , ".join(cell[0]) for cell in table['header']]

        answer_nodes = d['answer-node']
        answer_rows = set([_[1][0] for _ in answer_nodes])

        tmp = []
        labels = []
        for node in d['tf-idf']:
            tmp.append(node + [headers[node[1][1]], 'tf-idf'])        
            if node[1][0] in answer_rows:
                labels.append(1)
            else:
                labels.append(0)

        for node in d['string-overlap']:
            tmp.append(node + [headers[node[1][1]], 'string-overlap'])        
            if node[1][0] in answer_rows:
                labels.append(1)
            else:
                labels.append(0)

        for node in d['links']:
            tmp.append(node + [headers[node[1][1]], 'links'])   
            if node[1][0] in answer_rows:
                labels.append(1)
            else:
                labels.append(0)
        
        if d['question_id'] not in dev_keys and d['question_id'] not in test_keys:
            train_split.append({'question': d['question'], 'question_id': d['question_id'], 'table_id': d['table_id'], 
                              'nodes': tmp, 'labels': labels})
        elif d['question_id'] in dev_keys:
            dev_split.append({'question': d['question'], 'question_id': d['question_id'], 'table_id': d['table_id'], 
                              'nodes': tmp, 'labels': labels})

with open('Mixed-Reasoning/stage1_training_data.json', 'w') as f:
    json.dump(train_split, f, indent=2)

with open('Mixed-Reasoning/stage1_dev_data.json', 'w') as f:
    json.dump(dev_split, f, indent=2)
    
print("Done with Stage1 Data Processing")

In [ ]:
from multiprocessing import Pool

def func(d):
    train_split, dev_split = [], []
    if d['type'] in ['medium', 'easy']:
        table_id = d['table_id']
        with open('tables_tok/{}.json'.format(table_id), 'r') as f:
            table = json.load(f)
        
        with open('request_tok/{}.json'.format(table_id), 'r') as f:
            requested_document = json.load(f)
        
        headers = [cell[0][0] for cell in table['header']]
        
        answer_nodes = d['answer-node']
        answer_rows = {_[1][0]: _ for _ in answer_nodes}

        labels = []
        for name, source in zip(['tf-idf', 'string-overlap', 'links'], [d['tf-idf'], d['string-overlap'], d['links']]):
            for node in source:
                i = node[1][0]
                if i in answer_rows and i >= 0:
                    tmp = {'question': d['question'], 'question_id': d['question_id'], 'table_id': d['table_id'], 'current': node + [headers[node[1][1]], name]}
                    target_nodes = []
                    labels = []
                    same_row = table['data'][i]
                    for j, cell in enumerate(same_row):
                        for content, url in zip(cell[0], cell[1]):
                            if len(content) > 0:
                                if url:
                                    doc = requested_document[url]
                                    intro = filter_firstKsents(doc, 1)
                                    target_nodes.append((content, (i, j), url, headers[j], intro))
                                    if url == answer_rows[i][2]:
                                        labels.append(1)
                                    else:
                                        labels.append(0)
                                else:
                                    target_nodes.append((content, (i, j), None, headers[j], ''))
                                    if content == answer_rows[i][0]:
                                        labels.append(1)
                                    else:
                                        labels.append(0)
                                
                        if len(cell[0]) > 1:
                            content = ' , '.join(cell[0])
                            if content == answer_rows[i][0]:
                                labels.append(1)
                            else:
                                labels.append(0)
                                
                            target_nodes.append((content, (i, j), None, headers[j], ''))
                        
                    tmp['labels'] = labels

                    assert sum(labels) > 0, d['question_id']
                    
                    tmp['target'] = target_nodes

                    if tmp['question_id'] not in dev_keys and tmp['question_id'] not in test_keys:
                        train_split.append(tmp)
                    elif tmp['question_id'] in dev_keys:
                        dev_split.append(tmp)

    return train_split, dev_split


pool = Pool(64)
results = pool.map(func, data)

train_split = []
dev_split = []
for r1, r2 in results:
    train_split.extend(r1)
    dev_split.extend(r2)

with open('Mixed-Reasoning/stage2_training_data.json', 'w') as f:
    json.dump(train_split, f, indent=2)

with open('Mixed-Reasoning/stage2_dev_data.json', 'w') as f:
    json.dump(dev_split, f, indent=2)

print("Done with Stage2 Data Processing")

In [ ]:
train_split = []
dev_split = []
inside, self = 0, 0
for d in data:
    if d['where'] == 'passage':
        table_id = d['table_id']
        
        with open('request_tok/{}.json'.format(table_id)) as f:
            requested_documents = json.load(f)        
        
        #tmp = mapping.get(str(table_id), [])
        
        used = set()
        for node in d['answer-node']:
            if node[2] not in used:
                context = requested_documents[node[2]]
                context = 'Title : {} . '.format(node[0]) + context
                
                orig_answer = d['answer-text']

                start = context.lower().find(orig_answer.lower())

                if start == -1:
                    import pdb
                    pdb.set_trace()

                while context[start].lower() != orig_answer[0].lower():
                    start -= 1

                answer = context[start:start+len(orig_answer)]
                #assert(answer.lower() == orig_answer.lower(), "{} -> {}".format(answer, orig_answer))
                
                if d['question_id'] not in dev_keys and d['question_id'] not in test_keys:
                    train_split.append({'context': context, 'title': table_id, 
                                      'question': d['question'], 'question_id': d['question_id'],
                                      'answers': [{'answer_start': start, 'text': answer}]})
                elif d['question_id'] in dev_keys:
                    dev_split.append({'context': context, 'title': table_id, 
                                      'question': d['question'], 'question_id': d['question_id'],
                                      'answers': [{'answer_start': start, 'text': answer}]})
                inside += 1
                used.add(node[2])
            else:
                continue
    
    if d['where'] == 'table':
        table_id = d['table_id']
        
        with open('request_tok/{}.json'.format(table_id)) as f:
            requested_documents = json.load(f)  
            
        used = set()
        for node in d['answer-node']:
            if node[2] and node[2] not in used:
                context = requested_documents[node[2]]
                context = 'Title : {} . '.format(node[0]) + context
                
                orig_answer = node[0]

                start = context.lower().find(orig_answer.lower())

                if start == -1:
                    import pdb
                    pdb.set_trace()

                while context[start].lower() != orig_answer[0].lower():
                    start -= 1
                    
                answer = context[start:start+len(orig_answer)]
                
                if d['question_id'] not in dev_keys and d['question_id'] not in test_keys:
                    train_split.append({'context': context, 'title': table_id, 
                                      'question': d['question'], 'question_id': d['question_id'],
                                      'answers': [{'answer_start': start, 'text': answer}]})
                elif d['question_id'] in dev_keys:
                    dev_split.append({'context': context, 'title': table_id, 
                                      'question': d['question'], 'question_id': d['question_id'],
                                      'answers': [{'answer_start': start, 'text': answer}]})
                self += 1
                used.add(node[2])
            else:
                continue

with open('Mixed-Reasoning/stage3_training_data.json', 'w') as f:
    json.dump(train_split, f, indent=2)
    
with open('Mixed-Reasoning/stage3_dev_data.json', 'w') as f:
    json.dump(dev_split, f, indent=2)

#print("Total amount of training instance = {} and dev instance = {}".format(len(training_data), len(dev_data)))
print("Looking inside the passage = {} and self loop = {}".format(inside, self))

# Transforming all files into gzip

In [ ]:
from utils import *

fs = ['Mixed-Reasoning/stage1_training_data.json', 'Mixed-Reasoning/stage1_dev_data.json',
    'Mixed-Reasoning/stage2_training_data.json', 'Mixed-Reasoning/stage2_dev_data.json', 
    'Mixed-Reasoning/stage3_training_data.json', 'Mixed-Reasoning/stage3_dev_data.json']

for f_n in fs:
    compressGZip(f_n)

In [ ]:
with open('Mixed-Reasoning/stage1_training_data.json', 'r') as f:
    data = json.load(f)
    
distribution = []
for d in data:
    distribution.append(len(d['nodes']))
    
import matplotlib.pyplot as plt

n, bins, patches = plt.hist(distribution, 100, facecolor='blue', alpha=0.5)
plt.show()

# Generating data for KBQA method

In [ ]:
import json

with open('Mixed-Reasoning/processed_step2.json', 'r') as f:
    data = json.load(f)

with open('Mixed-Reasoning/dev_ids.json', 'r') as f:
    dev_ids = set(json.load(f))

with open('Mixed-Reasoning/test_ids.json', 'r') as f:
    test_ids = set(json.load(f))

print("there are {} dev and {} testing instances".format(len(dev_ids), len(test_ids)))
def hash_string(string):
    import hashlib
    sha = hashlib.sha256()
    sha.update(string.encode())
    return sha.hexdigest()[:16]
    
new_data = []
for p in data:
    if p['answer-node'] and p['answer-node'][0][-1] == 'table':
        if len(p['answer-node']) > 1 and p['answer-node'][0][-1] == 'table':
            with open('tables_tok/{}.json'.format(p['table_id']), 'r') as f:
                table = json.load(f)
            headers = [" , ".join(cell[0]) for cell in table['header']]    

            potential_headers = set()
            for h in headers:
                if " " + h.lower() + " " in " " + p['question'].lower() + " ":
                    potential_headers.add(h)

            if len(potential_headers) > 0:
                remaining_nodes = []
                for n in p['answer-node']:
                    if headers[n[1][1]] in potential_headers:
                        remaining_nodes.append(n)

                if len(remaining_nodes) > 0 and len(remaining_nodes) < len(p['answer-node']):
                    p['answer-node'] = remaining_nodes    
        
        del p['string-overlap']
        del p['tf-idf']
        
        if len(p['links']) > 0:
            new_data.append(p)

print("totally {} instances".format(len(new_data)))

outputs = []
for d in new_data:
    question_id = hash_string(d['question'])
    if question_id in dev_ids or question_id in test_ids:
        #print("IN DEV/TEST")
        continue
    
    table_id = d['table_id']
    with open('tables_tok/{}.json'.format(table_id)) as f:
        table = json.load(f)
    headers = [_[0][0] for _ in table['header']]
    
    answer_mapping = {}
    for node in d['answer-node']:
        answer_mapping[node[1][0]] = [headers[node[1][1]]]
    
    for link in d['links']:
        if link[1][0] in answer_mapping:
            answer_mapping[node[1][0]].append(link)
    
    potential_link = []
    for link in d['links']:
        if (headers[link[1][1]], link[0]) not in potential_link:
            if link[3] == 'string match':
                potential_link.append((headers[link[1][1]], link[0]))
    potential_link.append(('*', '*'))
    
    for k, v in answer_mapping.items():
        if len(v) > 1:
            tmp = {'aggregate': '', 'target': None, 'where': []}
            
            for link in v[1:]:
                if link[3] != 'string match':
                    tmp['aggregate'] = link[3]
                    tmp['target'] = v[0]
                else:
                    tmp['target'] = v[0]
                    if (headers[link[1][1]], link[0]) not in tmp['where']:
                        tmp['where'].append((headers[link[1][1]], link[0]))
            
            if len(tmp['where']) > 0:
                tmp['where'] = tmp['where'][0]
            else:
                tmp['where'] = ['*', '*']
            
            outputs.append({'question': d['question'], 'table_id': d['table_id'],
                            'question_id': question_id, 'answer-text': d['answer-text'],
                            'groundtruth': tmp, 'aggregate': ['maximum', 'minimum', 'earliest', 'latest', ''], 
                            'target': headers, 'where': potential_link})

with open('Mixed-Reasoning/KBQA_step1.json', 'w') as f:
    json.dump(outputs, f, indent=2)

In [ ]:
with open('Mixed-Reasoning/dev_input.json') as f:
    data = json.load(f)

possible = 0
outputs = []
for d in data:
    table_id = d['table_id']
    with open('tables_tok/{}.json'.format(table_id)) as f:
        table = json.load(f)
    headers = [_[0][0] for _ in table['header']]
    
    potential_link = []
    for n in d['nodes']:
        if n[-1] == 'links' and n[3] == 'string match':
            potential_link.append((headers[n[1][1]], n[0]))
    
    potential_link.append(('*', '*'))
    
    if len(potential_link) > 1:
        possible += 1        
    
    outputs.append({'question': d['question'], 'table_id': d['table_id'],
                    'question_id': d['question_id'], 'answer-text': d['answer-text'],
                    'aggregate': ['maximum', 'minimum', 'earliest', 'latest', ''], 
                    'target': headers, 'where': potential_link})

with open('Mixed-Reasoning/dev_KBQA_step1.json', 'w') as f:
    json.dump(outputs, f, indent=2)
    
print("upper bound of {}/{}".format(possible, len(data)))

# Statistics of the table/passages.

In [ ]:
column_num = 0
row_num = 0
cell_length = 0

total = 13000
total_cell = 0
total_url = 0
for i in range(0, total):
    with open('tables_tok/{}.json'.format(i)) as f:
        table = json.load(f)
    column_num += len(table['header'])
    row_num += len(table['data'])
    
    for row in table['data']:
        for cell in row:
            content = ' , '.join(cell[0])
            cell_length += len(content.split(' '))
            total_cell += 1
            
            for url in cell[1]:
                if url:
                    total_url += 1
    
print('column num = {}; row num = {}'.format(column_num / total, row_num / total))
print('cell num = {}; average length/cell = {}; average url/table = {}'.format(
    total_cell / total, cell_length / total_cell, total_url / total))

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

with open('wikipedia/merged_unquote.json', 'r') as f:
    passages = json.load(f)

vs = []
for k, v in passages.items():
    vs.append(tokenizer.tokenize(v)[:12])

In [ ]:
sent_length, word_length = 0, 0
for v in vs:
    sent_length += len(v)
    word_length += len((" ".join(v)).split(' '))

print("sentence length = {}, word length = {}".format(sent_length / len(vs), word_length / len(vs)))

In [ ]:
with open('Mixed-Reasoning/processed_step3.json', 'r') as f:
    data = json.load(f)

length = 0
for d in data:
    length += len(d['question'].split(' '))

print("avg length = {}".format(length / len(data)))

with open('Mixed-Reasoning/dev_ids.json') as f:
    dev_ids = set(json.load(f))
    
with open('Mixed-Reasoning/test_ids.json') as f:
    test_ids = set(json.load(f))

tr_easy, tr_med, tr_hard, tr_cell, tr_passage = 0, 0, 0, 0, 0
dev_easy, dev_med, dev_hard, dev_cell, dev_passage  = 0, 0, 0, 0, 0
test_easy, test_med, test_hard, test_cell, test_passage = 0, 0, 0, 0, 0

for d in data:
    if d['question_id'] not in dev_ids and d['question_id'] not in test_ids:
        if d['where'] == 'table':
            tr_cell += 1
        else:
            tr_passage += 1
        
        if d['type'] == 'easy':
            tr_easy += 1
        elif d['type'] in ['medium', 'numeric']:
            tr_med += 1
        else:
            tr_hard += 1
    elif d['question_id'] in dev_ids:
        if d['where'] == 'table':
            dev_cell += 1
        else:
            dev_passage += 1
        
        if d['type'] == 'easy':
            dev_easy += 1
        elif d['type'] in ['medium', 'numeric']:
            dev_med += 1
        else:
            dev_hard += 1
    elif d['question_id'] in test_ids:
        if d['where'] == 'table':
            test_cell += 1
        else:
            test_passage += 1
        
        if d['type'] == 'easy':
            test_easy += 1
        elif d['type'] in ['medium', 'numeric']:
            test_med += 1
        else:
            test_hard += 1

print(tr_easy, tr_med, tr_hard, tr_cell, tr_passage)
print(dev_easy, dev_med, dev_hard, dev_cell, dev_passage)
print(test_easy, test_med, test_hard, test_cell, test_passage)

# Miscelleous

In [ ]:
import json

def is_num(num):
    try:
        float(num)
        return True
    except Exception:
        return False

plausible_tables = []
for i in range(7000, 15000):
    with open('tables/{}.json'.format(i)) as f:
        table = json.load(f)
    
    for column_idx in range(len(table['header'])):      
        if all([is_num(table['data'][row_idx][column_idx][0][0]) for row_idx in range(len(table['data']))]):    
            plausible_tables.append(i)
            break
            
print(len(plausible_tables))

with open('Mixed-Reasoning/numeric_tables.json', 'w') as f:
    json.dump(plausible_tables, f, indent=2)

with open('Mixed-Reasoning/processed_step2.json', 'r') as f:
    data = json.load(f)
    
mismatch = 0
match = 0
for d in data:
    if d['where'] == 'table':
        if d['answer-text'] != d['answer-node'][0][0]:
            print(d['answer-node'][0][0], '#', d['answer-text'])
            mismatch += 1
        else:
            match += 1

print("final match = {}, partial match = {}".format(match, mismatch)) 